### Google Colab Requirements

In [2]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q faster-whisper yt-dlp
!sudo apt-get update && sudo apt-get install -y ffmpeg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.3 MB/s

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Main Code

In [4]:
import os
import subprocess
import json
import re
import sys
import time
from faster_whisper import WhisperModel

def sanitize_filename(filename):
    """파일 이름으로 사용할 수 없는 문자를 제거하는 함수"""
    return re.sub(r'[\\/*?:"<>|]', "", filename)

def get_all_video_urls(channel_url):
    """yt-dlp를 사용해 채널의 모든 영상 URL을 가져오는 함수"""
    print(f"채널에서 모든 영상 URL을 가져오는 중입니다...")
    try:
        command = ['yt-dlp', '--flat-playlist', '--print', 'webpage_url', channel_url]
        result = subprocess.run(
            command, capture_output=True, text=True, check=True, encoding='utf-8'
        )
        urls = [line for line in result.stdout.strip().split('\n') if line.strip() and line.startswith('http')]
        print(f"총 {len(urls)}개의 유효한 영상을 찾았습니다.")
        return urls
    except subprocess.CalledProcessError as e:
        print(f"URL 목록을 가져오는 데 실패했습니다. yt-dlp 오류:\n{e.stderr}")
        return []
    except Exception as e:
        print(f"URL 목록 처리 중 예기치 않은 오류 발생: {e}")
        return []

def get_video_info(url):
    """yt-dlp를 사용해 영상의 제목과 ID를 가져오는 함수"""
    try:
        command = ['yt-dlp', '--print', '{"title": "%(title)s", "id": "%(id)s"}', '--skip-download', url]
        process = subprocess.run(command, capture_output=True, text=True, check=True, encoding='utf-8')
        info = json.loads(process.stdout.strip().split('\n')[-1])
        return info['title'], info['id']
    except Exception as e:
        print(f"영상 정보({url})를 가져오는 데 실패했습니다: {e}")
        return None, None

def download_audio(url, video_id, temp_dir):
    """yt-dlp를 사용해 오디오를 다운로드하는 함수"""
    try:
        output_template = os.path.join(temp_dir, f"{video_id}.%(ext)s")
        command = [
            'yt-dlp',
            '--format', 'bestaudio/best',
            '--extract-audio',
            '--audio-format', 'm4a',
            '--output', output_template,
            '--no-mtime',
            url
        ]

        print(f"'{video_id}' 오디오 다운로드 중...")
        subprocess.run(command, check=True, capture_output=True, text=True, encoding='utf-8')

        expected_path = os.path.join(temp_dir, f"{video_id}.m4a")
        time.sleep(1)

        if not os.path.exists(expected_path):
            raise FileNotFoundError(f"다운로드 후 파일을 찾을 수 없습니다: {expected_path}")

        return expected_path

    except subprocess.CalledProcessError as e:
        print(f"오디오 다운로드 중 오류 발생: yt-dlp가 오류 코드를 반환했습니다.")
        print("---------- yt-dlp STDERR (오류 시) ----------")
        print(e.stderr)
        print("-----------------------------------")
        return None
    except Exception as e:
        print(f"오디오 다운로드 처리 중 예기치 않은 오류 발생: {e}")
        return None

In [ ]:
# --- 설정 (사용자 지정 가능) ---
CHANNEL_URL = "https://www.youtube.com/@hongcha_._/videos"

# 결과를 저장할 폴더 경로
OUTPUT_DIR = "/content/drive/MyDrive/HONGCHA_SCRIPTS"

# Whisper 모델 설정
MODEL_SIZE = "medium"
DEVICE = "cuda"
COMPUTE_TYPE = "float16"
# -----------------------------------

# 디렉토리 설정
temp_dir_abs = "/content/temp_audio_files"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(temp_dir_abs, exist_ok=True)

print("--- 자동 음성 변환 스크립트 시작 ---")
print("Whisper 모델을 로드합니다...")
try:
    model = WhisperModel(MODEL_SIZE, device=DEVICE, compute_type=COMPUTE_TYPE)
except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")
    raise e

video_urls = get_all_video_urls(CHANNEL_URL)
if not video_urls:
    print("처리할 영상이 없습니다. 스크립트를 종료합니다.")
else:
    for i, url in enumerate(video_urls):
        print(f"\n--- [{i+1}/{len(video_urls)}] 영상 처리 시작: {url} ---")

        downloaded_audio_path = None
        try:
            video_title, video_id = get_video_info(url)
            if not video_title or not video_id:
                continue

            print(f"영상 제목: {video_title}")
            sanitized_title = sanitize_filename(video_title)
            output_filepath = os.path.join(OUTPUT_DIR, f"{sanitized_title}.txt")

            if os.path.exists(output_filepath):
                print(f"'{sanitized_title}.txt' 파일이 이미 존재하므로 건너뜁니다.")
                continue

            downloaded_audio_path = download_audio(url, video_id, temp_dir_abs)
            if not downloaded_audio_path:
                continue

            print(f"오디오 파일 경로: {downloaded_audio_path}")

            device_name = "GPU" if DEVICE == "cuda" else "CPU"
            print(f"음성 인식 변환 중 ({device_name} 사용)...")
            segments, info = model.transcribe(downloaded_audio_path, beam_size=5, language="ko")

            transcript = "".join([segment.text for segment in segments]).strip()
            print("음성 인식 완료.")

            with open(output_filepath, "w", encoding="utf-8") as f:
                f.write(transcript)
            print(f"대본 저장 완료: {output_filepath}")

        except Exception as e:
            print(f"영상 처리 루프 중 예기치 않은 오류 발생: {e}")

        finally:
            if downloaded_audio_path and os.path.exists(downloaded_audio_path):
                try:
                    os.remove(downloaded_audio_path)
                    print("임시 오디오 파일 삭제 완료.")
                except OSError as e:
                    print(f"임시 파일 삭제 실패: {e}")

print("\n--- 모든 작업이 완료되었습니다. ---")

--- 자동 음성 변환 스크립트 시작 ---
Whisper 모델을 로드합니다... (T4 GPU 사용)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

채널에서 모든 영상 URL을 가져오는 중입니다...
총 1093개의 유효한 영상을 찾았습니다.

--- [1/1093] 영상 처리 시작: https://www.youtube.com/watch?v=zjPgj1vMVCg ---
영상 제목: 연알못 남자와 사귀는 여자의 마음
'zjPgj1vMVCg' 오디오 다운로드 중...
오디오 파일 경로: /content/temp_audio_files/zjPgj1vMVCg.m4a
음성 인식 변환 중 (GPU 사용)...
음성 인식 완료.
대본 저장 완료: /content/drive/MyDrive/HONGCHA_SCRIPTS/연알못 남자와 사귀는 여자의 마음.txt
임시 오디오 파일 삭제 완료.

--- [2/1093] 영상 처리 시작: https://www.youtube.com/watch?v=No1x9wkWB2A ---
영상 제목: 남자를 좋아할 때 여자가 질투나는 순간
'No1x9wkWB2A' 오디오 다운로드 중...
오디오 파일 경로: /content/temp_audio_files/No1x9wkWB2A.m4a
음성 인식 변환 중 (GPU 사용)...
음성 인식 완료.
대본 저장 완료: /content/drive/MyDrive/HONGCHA_SCRIPTS/남자를 좋아할 때 여자가 질투나는 순간.txt
임시 오디오 파일 삭제 완료.

--- [3/1093] 영상 처리 시작: https://www.youtube.com/watch?v=AlYbWvMhJPk ---
영상 제목: 잘생겨도 여자가 안만나는 남자 특징
'AlYbWvMhJPk' 오디오 다운로드 중...
오디오 파일 경로: /content/temp_audio_files/AlYbWvMhJPk.m4a
음성 인식 변환 중 (GPU 사용)...
음성 인식 완료.
대본 저장 완료: /content/drive/MyDrive/HONGCHA_SCRIPTS/잘생겨도 여자가 안만나는 남자 특징.txt
임시 오디오 파일 삭제 완료.

--- [4/1093] 영상 처리 시작: https://www.

### Note

* 코드 실행 중 Colab 무료 T4 사용량을 초과하여 런타임이 종료될 수 있습니다.

* 이에 언제든 끊어졌던 부분부터 다시 시작할 수 있도록 URL 목록이 담긴 `.txt` 파일을 생성합니다.

In [7]:
CHANNEL_URL = "https://www.youtube.com/@hongcha_._/videos"
OUTPUT_DIR = "/content/drive/MyDrive/HONGCHA_SCRIPTS/LIST"
hongcha_list = get_all_video_urls(CHANNEL_URL)
with open("hongcha_list.txt", "w") as f:
    for i in hongcha_list:
        f.write(f"{i}\n")

채널에서 모든 영상 URL을 가져오는 중입니다...
총 1093개의 유효한 영상을 찾았습니다.
